In [ ]:
"""
Advanced Tax Assistant System
- Implements US 2024 Tax Rules
- Uses ML for Deduction Optimization
- Full Input Validation
- Type-Safe Calculations
"""

from typing import Dict, Literal, Tuple
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Type definitions
FilingStatus = Literal['single', 'married']
TaxBrackets = Dict[FilingStatus, Tuple[float, ...]]
TaxRates = Tuple[float, ...]

class TaxError(Exception):
    """Base class for tax calculation exceptions"""

class InvalidIncomeError(TaxError):
    """Raised for invalid income values"""

class InvalidFilingStatusError(TaxError):
    """Raised for invalid filing status"""

class DeductionPredictionError(TaxError):
    """Raised for deduction prediction failures"""

class TaxConfig:
    """Tax configuration constants"""
    BRACKETS: TaxBrackets = {
        'single': (0, 11000, 44725, 95375, 182100, 231251),
        'married': (0, 22000, 89450, 190750, 364200, 462500)
    }
    RATES: TaxRates = (0.10, 0.12, 0.22, 0.24, 0.32, 0.35)
    STANDARD_DEDUCTIONS = {
        'single': 13850,
        'married': 27700
    }
    MEDICAL_FLOOR_PCT = 0.075  # 7.5% AGI floor for medical expenses

class TaxCalculator:
    """Handles progressive tax calculations with full validation"""

    def __init__(self, filing_status: FilingStatus):
        self.filing_status = filing_status
        self._validate_config()

    def calculate(self, taxable_income: float) -> float:
        """
        Calculate tax liability using progressive brackets
        Args:
            taxable_income: Pre-deduction gross income
        Returns:
            Rounded tax amount
        """
        self._validate_income(taxable_income)

        tax = 0.0
        remaining = taxable_income
        brackets = sorted(TaxConfig.BRACKETS[self.filing_status], reverse=True)

        for bracket, rate in zip(brackets, TaxConfig.RATES):
            if remaining > bracket:
                tax += (remaining - bracket) * rate
                remaining = bracket
        return round(tax, 2)

    def _validate_config(self) -> None:
        """Validate tax configuration integrity"""
        if len(TaxConfig.BRACKETS[self.filing_status]) != len(TaxConfig.RATES):
            logging.error("Tax configuration mismatch")
            raise ValueError("Brackets and rates length mismatch")

    def _validate_income(self, income: float) -> None:
        """Validate income value"""
        if income < 0:
            logging.error(f"Negative income: {income}")
            raise InvalidIncomeError("Income cannot be negative")

class DeductionOptimizer:
    """Machine learning model for deduction optimization"""

    def __init__(self):
        self.model = self._init_model()
        self._train_model()

    def predict(self, features: np.ndarray) -> bool:
        """
        Predict optimal deduction strategy
        Args:
            features: [income, mortgage, charity, medical, is_married]
        Returns:
            True if itemizing deductions is better
        """
        if features.shape != (5,):
            logging.error(f"Invalid feature shape: {features.shape}")
            raise DeductionPredictionError("Invalid feature dimensions")

        try:
            return bool(self.model.predict([features])[0])
        except Exception as e:
            logging.error(f"Prediction failed: {str(e)}")
            raise DeductionPredictionError("ML prediction error") from e

    def _init_model(self) -> Pipeline:
        """Initialize ML pipeline"""
        return Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', DecisionTreeClassifier(
                max_depth=4,
                min_samples_leaf=50,
                class_weight='balanced',
                random_state=42
            ))
        ])

    def _train_model(self) -> None:
        """Train model with synthetic data"""
        X = np.array([
            [75_000, 12_000, 2_500, 5_000, 0],  # Single, itemize
            [45_000, 0, 500, 1_000, 0],        # Single, standard
            [150_000, 25_000, 5_000, 15_000, 1], # Married, itemize
            [80_000, 5_000, 1_000, 3_000, 0]   # Single, standard
        ])
        y = np.array([1, 0, 1, 0])
        self.model.fit(X, y)

class TaxAssistant:
    """Main tax processing system with comprehensive validation"""

    def __init__(self):
        self.deduction_optimizer = DeductionOptimizer()
        self.logger = logging.getLogger(__name__)

    def process_filing(self, user_input: Dict) -> Dict:
        """
        Complete tax processing pipeline
        Args:
            user_input: Validated user data
        Returns:
            Complete tax results
        """
        self._validate_input(user_input)

        try:
            # Prepare ML features
            features = self._create_features(user_input)

            # Calculate deductions
            itemize = self.deduction_optimizer.predict(features)
            deductions = self._calculate_deductions(user_input, itemize)

            # Calculate taxable income
            taxable_income = self._calculate_taxable_income(
                user_input['income'],
                deductions
            )

            # Calculate tax
            tax_owed = TaxCalculator(user_input['filing_status']).calculate(taxable_income)

            return {
                'gross_income': user_input['income'],
                'deductions': deductions,
                'taxable_income': taxable_income,
                'tax_owed': tax_owed,
                'filing_status': user_input['filing_status'],
                'itemized': itemize
            }

        except Exception as e:
            self.logger.error(f"Processing failed: {str(e)}")
            raise

    def _create_features(self, user_input: Dict) -> np.ndarray:
        """Create ML features array"""
        return np.array([
            user_input['income'],
            user_input['mortgage_interest'],
            user_input['charity_donations'],
            max(0, user_input['medical_expenses'] -
                user_input['income'] * TaxConfig.MEDICAL_FLOOR_PCT),
            1 if user_input['filing_status'] == 'married' else 0
        ])

    def _calculate_deductions(self, user_input: Dict, itemize: bool) -> float:
        """Calculate total deductions"""
        if itemize:
            return sum([
                user_input['mortgage_interest'],
                user_input['charity_donations'],
                max(0, user_input['medical_expenses'] -
                    user_input['income'] * TaxConfig.MEDICAL_FLOOR_PCT)
            ])
        return TaxConfig.STANDARD_DEDUCTIONS[user_input['filing_status']]

    def _calculate_taxable_income(self, gross: float, deductions: float) -> float:
        """Calculate taxable income with validation"""
        taxable = gross - deductions
        if taxable < 0:
            self.logger.warning("Taxable income below zero - clamping to zero")
            return 0.0
        return taxable

    def _validate_input(self, user_input: Dict) -> None:
        """Validate all user inputs"""
        required = ['filing_status', 'income', 'mortgage_interest',
                   'charity_donations', 'medical_expenses']
        missing = [field for field in required if field not in user_input]
        if missing:
            raise ValueError(f"Missing fields: {missing}")

        if user_input['filing_status'] not in TaxConfig.BRACKETS:
            raise InvalidFilingStatusError("Invalid filing status")

        for field in ['income', 'mortgage_interest', 'charity_donations', 'medical_expenses']:
            if user_input[field] < 0:
                raise ValueError(f"Negative value not allowed for {field}")

def get_user_input() -> Dict:
    """Collect and validate user input with error handling"""
    print("\n=== Tax Filing Input ===")
    while True:
        try:
            return {
                'filing_status': input("Filing status (single/married): ").strip().lower(),
                'income': float(input("Gross annual income ($): ")),
                'mortgage_interest': float(input("Mortgage interest paid ($): ")),
                'charity_donations': float(input("Charitable donations ($): ")),
                'medical_expenses': float(input("Medical expenses ($): "))
            }
        except ValueError as e:
            print(f"Invalid input: {str(e)}. Please try again.")

def format_currency(amount: float) -> str:
    """Format numbers as currency strings"""
    return f"${amount:,.2f}"

if __name__ == "__main__":
    try:
        user_data = get_user_input()
        assistant = TaxAssistant()
        results = assistant.process_filing(user_data)

        print("\n=== Tax Calculation Results ===")
        print(f"Filing Status: {results['filing_status'].title()}")
        print(f"Gross Income: {format_currency(results['gross_income'])}")
        print(f"Deductions ({'Itemized' if results['itemized'] else 'Standard'}): "
              f"{format_currency(results['deductions'])}")
        print(f"Taxable Income: {format_currency(results['taxable_income'])}")
        print(f"Estimated Tax Owed: {format_currency(results['tax_owed'])}")

    except TaxError as e:
        print(f"\nError: {str(e)}")
        print("Please check your inputs and try again.")
    except Exception as e:
        print(f"\nUnexpected error: {str(e)}")
        print("Please contact support.")



=== Tax Filing Input ===
